In [5]:
# Lab 7 Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt # 설치해야함 
tf.set_random_seed(777)  # for reproducibility

ModuleNotFoundError: No module named 'matplotlib'

In [3]:

from tensorflow.examples.tutorials.mnist import input_data
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
# 0에서 9까지의 결과값이 나옴
nb_classes = 10

# MNIST data image of shape 28 * 28 = 784 -> 데이터는 비트맵으로 되어있음
X = tf.placeholder(tf.float32, [None, 784])
# 0 - 9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None, nb_classes])

W = tf.Variable(tf.random_normal([784, nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))


In [ ]:

# Hypothesis (using softmax)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)


In [ ]:

# Test model
# one hot 값과 model의 값을 비교한다.
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))


In [ ]:

# parameters
# 엄청난 크기의 데이터를 등분하여 처리함
# 전체 데이터를 한번 학습 시키는것을 epochs이라고 한다.
# 1 epochs을 batch_size로 분해하여 돈다.
# 예를 들어 전체 데이터가 1000개이고 batch_size가 500이면 1 epochs을 위해 2번의 작업 반복이 필요
training_epochs = 15
batch_size = 100


In [ ]:

with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)

        # i는 이터레이션, 전체 데이터 사이즈를 batch_size로 나누면 된다(tatal_batch).
        for i in range(total_batch):
            # batch_size 만큼 데이터를 훈련 시킴
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, optimizer], feed_dict={
                            X: batch_xs, Y: batch_ys})
            avg_cost += c / total_batch

        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.9f}'.format(avg_cost))

    print("Learning finished")

    # Test the model using test sets
    # 학습에 한번도 사용하지 않았던 테스트 데이터를 넣어줌
    # tensor에 eval이라는 함수를 호출하여 sess.run을 실행할 수 있음
    print("Accuracy: ", accuracy.eval(session=sess, feed_dict={
          X: mnist.test.images, Y: mnist.test.labels}))

    # Get one and predict
    r = random.randint(0, mnist.test.num_examples - 1)
    # one hot, 즉 결과값을 출력함, 인덱스는 0부터 이므로 1을 더함
    print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
    print("Prediction: ", sess.run(
        tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

    # 랜덤 이미지를 출력
    plt.imshow(
        mnist.test.images[r:r + 1].reshape(28, 28),
        cmap='Greys',
        interpolation='nearest')
    plt.show()
